## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[rep_mobile_citizens_id_city_partner]]

[[citizens_dir_mobile]]
___
### Table

In [23]:
query_text = """--sql
CREATE TABLE db1.new_users_pd_rep_mobile_total
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `new_created_account_day` UInt64,
    `new_created_account` UInt64,
    `new_activated_account_day` UInt64,
    `new_activated_account` UInt64
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [29]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.new_users_pd_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 30 MINUTE TO db1.new_users_pd_rep_mobile_total AS
        WITH t_created_at AS(
            SELECT
                report_date,
                partner_uuid,
                city,
                activated_at,
                created_at,
                citizen_id,
                ROW_NUMBER() OVER (
                            PARTITION BY
                                toStartOfMonth(report_date),
                                citizen_id
                            ORDER BY report_date
                ) = 1 AS is_first_mention_in_day
            FROM
                (SELECT
                    report_date,
                    partner_uuid,
                    city,
                    total_act.activated_at AS activated_at,
                    toDateOrNull(cit_dir_m.`created_at`) AS created_at,
                    citizen_id
                FROM db1.rep_mobile_citizens_id_city_partner AS total_act
                LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                            ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
                )
            WHERE created_at = report_date
        ),
        --
        t_activated_at AS(
            SELECT
                report_date,
                partner_uuid,
                city,
                activated_at,
                created_at,
                citizen_id,
                ROW_NUMBER() OVER (
                            PARTITION BY
                                toStartOfMonth(report_date),
                                citizen_id
                            ORDER BY report_date
                ) = 1 AS is_first_mention_in_day
            FROM
                (SELECT
                    report_date,
                    partner_uuid,
                    city,
                    toDate(total_act.activated_at) AS activated_at,
                    toDateOrNull(cit_dir_m.`created_at`) AS created_at,
                    citizen_id
                FROM db1.rep_mobile_citizens_id_city_partner AS total_act
                LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                            ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
                )
            WHERE activated_at = report_date
        ),
        --
        dec_created_at AS (SELECT
            DISTINCT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM t_created_at) AS t1
        CROSS JOIN
            (SELECT DISTINCT city, partner_uuid FROM t_created_at ) AS t2
        ),
        --
        dec_activated_at AS (SELECT
            DISTINCT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM t_activated_at) AS t1
        CROSS JOIN
            (SELECT DISTINCT city, partner_uuid FROM t_activated_at ) AS t2
        ),
        --
        dec_and_t_created_at AS (
            SELECT
                dec_created_at.report_date AS report_date,
                dec_created_at.partner_uuid AS partner_uuid,
                dec_created_at.city AS city,
                new_created_account_day
            FROM dec_created_at
            LEFT join (
                SELECT 
                    report_date,
                    partner_uuid,
                    city,
                    count(DISTINCT if(is_first_mention_in_day = 1,citizen_id,NULL)) AS new_created_account_day
                FROM t_created_at 
                GROUP BY
                    report_date,
                    partner_uuid,
                    city) AS full_table
                ON dec_created_at.report_date = full_table.report_date 
                AND dec_created_at.partner_uuid = full_table.partner_uuid
                AND dec_created_at.city = full_table.city
        ),
        --
        dec_and_t_activated_at AS (
            SELECT
                dec_activated_at.report_date AS report_date,
                dec_activated_at.partner_uuid AS partner_uuid,
                dec_activated_at.city AS city,
                new_activated_account_day
            FROM dec_activated_at
            LEFT join (
                SELECT 
                    report_date,
                    partner_uuid,
                    city,
                    count(DISTINCT if(is_first_mention_in_day = 1,citizen_id,NULL)) AS new_activated_account_day
                FROM t_activated_at 
                GROUP BY
                    report_date,
                    partner_uuid,
                    city
                    ) AS full_table
                ON dec_activated_at.report_date = full_table.report_date 
                AND dec_activated_at.partner_uuid = full_table.partner_uuid
                AND dec_activated_at.city = full_table.city
        ),
        --
        dec_and_t_activated_at_month AS (
            SELECT
                report_date,
                partner_uuid,
                city,
                new_activated_account_day,
                sum(new_activated_account_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS new_activated_account
            FROM dec_and_t_activated_at
        ),
        --
        dec_and_t_created_at_month AS (
            SELECT
                report_date,
                partner_uuid,
                city,
                new_created_account_day,
                sum(new_created_account_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS new_created_account
            FROM dec_and_t_created_at
        ),
        --
        t_dec AS(
        SELECT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM db1.installation_point_st_partner_ch) AS t1
        CROSS JOIN
            (SELECT DISTINCT city,inst.partner_uuid AS partner_uuid
            FROM db1.installation_point_st_partner_ch AS inst
            LEFT JOIN db1.entries_installation_points_dir_partner AS eipdp ON inst.installation_point_id  = eipdp.installation_point_id ) AS t2
        )
        --
        SELECT
            t_dec.report_date AS report_date,
            t_dec.partner_uuid AS partner_uuid,
            t_dec.city AS city,
            new_activated_account_day,
            new_activated_account,
            new_created_account_day,
            new_created_account
        FROM t_dec 
        LEFT JOIN dec_and_t_activated_at_month AS a_a_m 
            ON a_a_m.report_date = t_dec.report_date 
            AND a_a_m.city = t_dec.city
            AND a_a_m.partner_uuid = t_dec.partner_uuid
        LEFT JOIN dec_and_t_created_at_month AS c_a_m
            ON c_a_m.report_date = t_dec.report_date 
            AND c_a_m.city = t_dec.city
            AND c_a_m.partner_uuid = t_dec.partner_uuid
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [31]:
query_text = """--sql
    SELECT
        *
    FROM db1.new_users_pd_rep_mobile_total
    WHERE new_created_account !=0
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,new_created_account_day,new_created_account,new_activated_account_day,new_activated_account
date,str,str,i64,i64,i64,i64
2025-08-13,"""""","""""",278,329,405,405
2025-08-13,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",0,1,0,2
2025-08-13,"""010cd1e6-7f7b-4613-9209-96b28d…","""Котлас""",3,45,4,61
2025-08-13,"""00be0483-e667-4d7e-a462-64f1eb…","""Екатеринбург""",0,2,0,2
2025-08-13,"""00ec6d8e-a799-409c-958b-9472a1…","""Казань""",0,1,0,1
…,…,…,…,…,…,…
2025-08-13,"""42e30ded-03a9-43a1-9d16-46f4bd…","""Ростов-на-Дону""",0,1,0,4
2025-08-13,"""4416c289-92d3-4534-be67-218111…","""Улан-Удэ""",0,1,0,5
2025-08-13,"""4385a9e1-84a2-4090-aebf-4eff25…","""Ижевск""",1,5,1,14


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.nuw_users_pd_rep_mobile_total DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

### drop mv

In [28]:
query_text = """--sql
    DROP TABLE db1.new_users_pd_rep_mobile_total_mv
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.new_users_pd_rep_mobile_total_mv does not exist. Maybe you meant db1.nuw_users_pd_rep_mobile_total_mv?. (UNKNOWN_TABLE) (version 25.3.3.42 (official build))


### drop ch

In [22]:
query_text = """--sql
    DROP TABLE db1.new_users_pd_rep_mobile_total
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.nuw_users_pd_rep_mobile_total_mv
"""

ch.query_run(query_text)

In [ ]:
query_text = """--sql
        WITH t_activated_at AS(
            SELECT
                report_date,
                partner_uuid,
                city,
                activated_at,
                created_at,
                citizen_id,
                ROW_NUMBER() OVER (
                            PARTITION BY
                                toStartOfMonth(report_date),
                                citizen_id
                            ORDER BY report_date
                ) = 1 AS is_first_mention_in_day
            FROM
                (SELECT
                    report_date,
                    partner_uuid,
                    city,
                    toDate(total_act.activated_at) AS activated_at,
                    toDateOrNull(cit_dir_m.`created_at`) AS created_at,
                    citizen_id
                FROM db1.rep_mobile_citizens_id_city_partner AS total_act
                LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                            ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
                WHERE report_date BETWEEN '2025-07-01' AND '2025-07-31'
                )
            WHERE activated_at = report_date
        ),
        --
        dec_activated_at AS (SELECT
            DISTINCT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM t_activated_at) AS t1
        CROSS JOIN
            (SELECT DISTINCT city, partner_uuid FROM t_activated_at ) AS t2
        ),
        --
        dec_and_t_activated_at AS (
            SELECT
                dec_activated_at.report_date AS report_date,
                dec_activated_at.partner_uuid AS partner_uuid,
                dec_activated_at.city AS city,
                new_activated_account_day
            FROM dec_activated_at
            LEFT join (
                SELECT 
                    report_date,
                    partner_uuid,
                    city,
                    count(DISTINCT if(is_first_mention_in_day = 1,citizen_id,NULL)) AS new_activated_account_day
                FROM t_activated_at 
                GROUP BY
                    report_date,
                    partner_uuid,
                    city
                    ) AS full_table
                ON dec_activated_at.report_date = full_table.report_date 
                AND dec_activated_at.partner_uuid = full_table.partner_uuid
                AND dec_activated_at.city = full_table.city
        ),
        --
        dec_and_t_activated_at_month AS (
        SELECT
            report_date,
            partner_uuid,
            city,
            new_activated_account_day,
            sum(new_activated_account_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS new_activated_account
        FROM dec_and_t_activated_at
        )
        SELECT
            report_date,
            sum(new_activated_account)
        FROM dec_and_t_activated_at_month
        GROUP BY report_date
        ORDER BY report_date DESC
    """

ch.query_run(query_text)

report_date,sum(new_activated_account)
date,i64
2025-07-31,31780
2025-07-30,30717
2025-07-29,29633
2025-07-28,28608
2025-07-27,27562
…,…
2025-07-05,5718
2025-07-04,4773
2025-07-03,3638


In [38]:
query_text = """--sql
        WITH t_created_at AS(
            SELECT
                report_date,
                partner_uuid,
                city,
                activated_at,
                created_at,
                citizen_id,
                ROW_NUMBER() OVER (
                            PARTITION BY
                                toStartOfMonth(report_date),
                                citizen_id
                            ORDER BY report_date
                ) = 1 AS is_first_mention_in_day
            FROM
                (SELECT
                    report_date,
                    partner_uuid,
                    city,
                    total_act.activated_at AS activated_at,
                    toDateOrNull(cit_dir_m.`created_at`) AS created_at,
                    citizen_id
                FROM db1.rep_mobile_citizens_id_city_partner AS total_act
                LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                            ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
                WHERE report_date BETWEEN '2025-07-01' AND '2025-07-31'
                )
            WHERE created_at = report_date
        ),
        --
        dec_created_at AS (SELECT
            DISTINCT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM t_created_at) AS t1
        CROSS JOIN
            (SELECT DISTINCT city, partner_uuid FROM t_created_at ) AS t2
        ),
        --
        dec_and_t_created_at AS (
            SELECT
                dec_created_at.report_date AS report_date,
                dec_created_at.partner_uuid AS partner_uuid,
                dec_created_at.city AS city,
                new_created_account_day
            FROM dec_created_at
            LEFT join (
                SELECT 
                    report_date,
                    partner_uuid,
                    city,
                    count(DISTINCT if(is_first_mention_in_day = 1,citizen_id,NULL)) AS new_created_account_day
                FROM t_created_at 
                GROUP BY
                    report_date,
                    partner_uuid,
                    city) AS full_table
                ON dec_created_at.report_date = full_table.report_date 
                AND dec_created_at.partner_uuid = full_table.partner_uuid
                AND dec_created_at.city = full_table.city
        ),
        --
        dec_and_t_created_at_month AS (
        SELECT
            report_date,
            partner_uuid,
            city,
            sum(new_created_account_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS new_created_account
        FROM dec_and_t_created_at
        )
        SELECT
            report_date,
            sum(new_created_account)
        FROM dec_and_t_created_at_month
        GROUP BY report_date
        ORDER BY report_date DESC
    """

ch.query_run(query_text)

report_date,sum(new_created_account)
date,i64
2025-07-31,19522
2025-07-30,18835
2025-07-29,18170
2025-07-28,17582
2025-07-27,16885
…,…
2025-07-05,3520
2025-07-04,2935
2025-07-03,2245
